# TGBA-Practice Evaluation

## Initialization


In [ ]:
from pathlib import Path

Path("data/automata/practice").mkdir(parents=True, exist_ok=True)

Path("data/statistics/autcross").mkdir(parents=True, exist_ok=True)
Path("data/statistics/custom").mkdir(parents=True, exist_ok=True)

evalInput = "data/automata/practice/practice_automata.hoa"

autcrossOutFile = "data/statistics/autcross/TGBA_pract_tools.csv"
customOutFile = "data/statistics/custom/TGBA_pract_tools.csv"

In [ ]:
import os
SimCo = "python tools/SimCo/complement.py"
kofola = "tools/kofola/build/src/kofola"

os.environ["simco"] = SimCo
os.environ["kofola"] = kofola
os.environ["input"] = evalInput
os.environ["out"] = autcrossOutFile

### Statistics Generation Function

In [ ]:
import pandas as pd

def compare(file):
    data = pd.read_csv(file)

    tools = (set(data["tool"]))

    input_columns = [c for c in data.columns if c.startswith("input") and not c.endswith("name")]

    results = []

    data = data[data["input.acc_sets"] > 1] # Remove any TBAs if autfilt did not catch them
    
    for t in tools:
        print( f"Configuration {t} timed out ({len(data[(data['exit_status'] == 'timeout') & (data['tool'] == t)])})-times.")
        print( f"Configuration {t} did not finish due to error ({len(data[(data['exit_status'] != 'timeout') & (data['exit_status'] != 'ok') & (data['tool'] == t)])})-times.")
        
    for t in tools:
        t_result = data[data.tool == t]
        t_result = t_result.rename(columns={"exit_status" : f"{t}_exit_status"})
        t_result = t_result.rename(columns={"output.states" : t})
        results.append(t_result)
        
    data = data[input_columns].drop_duplicates()
    
    for t_result, t in zip(results, tools):
        data = data.merge(t_result[["input.source", t, f"{t}_exit_status"]], on="input.source", how="inner")
        data[t] = pd.to_numeric(data[t])
    
    for acc_sets in range(2, 5):
        filtered = data.dropna()
        filtered = filtered[filtered["input.acc_sets"] == acc_sets]

        print(f"Acc_sets = {acc_sets} | ", end="")
        for tool in tools:
            print(f"[{tool}] sum = {filtered[tool].sum():>{10}} | mean = {filtered[tool].mean():>{10}.2f}; std = {filtered[tool].std():>{10}.2f} | ", end="")
        print( )
        
    print("Total        | ", end="")
    for tool in tools:

        print(f"[{tool}] sum = {data.dropna()[tool].sum():>{10}} | mean = {data.dropna()[tool].mean():>{10}.2f}; std = {data.dropna()[tool].std():>{10}.2f} | ", end="")
        
    return data

## Filtering of Automata


In [ ]:
files = list(Path("./data/automata-benchmarks/omega/pecan/examples").glob("*.hoa"))
files += list(Path("./data/automata-benchmarks/omega/sturmian_words").glob("*.hoa"))

!rm -f {evalInput} 

for file in files:
    !autfilt -F "{file}" -v --is-empty | autfilt --acc-sets=2.. --remove-fin --tgba >> {evalInput}


## Evaluation

In [ ]:
!autcross --timeout=300 -F $input -t\
    '{SimCo}                $simco %H>%O'\
    '{SimCo+}               $simco --skip --jump --merge --scc-based %H | autfilt -b >%O'\
    '{Kofola}               $kofola --complement --tba %H>%O'\
    '{Seminator}            seminator --complement %H>%O'\
    '{Autfilt}              autfilt --complement -b %H>%O'\
--no-check --csv > $out

## Data Evaluation and Plot Generation

In [ ]:
data = compare(autcrossOutFile)
data.to_csv(customOutFile)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

tools = ["Autfilt", "Seminator", "Kofola"]
configurations = ["SimCo", "SimCo+"]

data = data.rename(columns={"input.acc_sets" : "Acceptance Sets"})

for tool in tools + configurations:
    data.loc[data[f"{tool}_exit_status"] == "timeout", tool] = int(10**6.5)
    data.loc[(data[f"{tool}_exit_status"] != "timeout") & (data[f"{tool}_exit_status"] != "ok"), tool] = int(10**6.7)
    
for configuration in configurations:
    for tool in tools:
        plt.xscale('log')
        plt.yscale('log')
        plt.plot([0,10**6.5],[0,10**6.5], color="black", alpha=0.3)
        
        plt.ylim(1e-0, 10**6.9)
        plt.xlim(1e-0, 10**6.9)
        
        plt.axhline(10**6.5, color="red", alpha=0.2, linestyle="--", label="Timeout")
        plt.axvline(10**6.5, color="red", alpha=0.2, linestyle="--")
        
        plt.axhline(10**6.7, color="red", alpha=0.2, linestyle=":", label="Error")
        plt.axvline(10**6.7, color="red", alpha=0.2, linestyle=":")

        plot = sns.scatterplot(data=data, x=tool, y=configuration, style="Acceptance Sets", hue="Acceptance Sets", palette="deep")
        fig = plot.get_figure()
        fig.set_size_inches(10,7)

        if tool == "Seminator":
            plot.set_xlabel("Seminator 2")

        handles, labels = plt.gca().get_legend_handles_labels()

        plt.legend(handles, ["Timeout", "Error"] + [f"{i} Acc. sets" for i in range(2, 12)], loc="center", bbox_to_anchor=(0.5, -0.20), ncol=4)
        plt.tight_layout()

        if configuration == "SimCo+":
            fig.savefig(f"data/plots/{configuration}-{tool}-Practice.svg", dpi=300)
        plt.show()